# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das acomodações
Neste notebook, coletarei dados de acomodações anunciadas de algumas fontes. 

In [1]:
import pandas as pd

In [2]:
ss_sites = pd.read_csv('../Data/Imobiliarias/imobiliarias.csv', squeeze=True)

## Teste estático com BeautifulSoup

In [ ]:
from urllib.request import urlopen
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
response = urlopen("url", context=context)
html_source = response.read()
soup = BeautifulSoup(html_source, 'html.parser')

## Coletando acomodações com Selenium

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from urllib.request import urlopen
import ssl
import hashlib
import webbrowser
import re
import time

In [64]:
def add_row_df(df, columns, row):
    row_sr = pd.Series(row, index=columns)
    row_df = pd.DataFrame([row_sr])
    df = pd.concat([row_df, df], ignore_index=True)
    return df

def hash_data(data):
    return hashlib.md5(data.encode()).hexdigest()

def goto_imovel_url(imovel_url):
    context = ssl._create_unverified_context()
    response = urlopen(imovel_url, context=context)
    html_source = response.read()
    return BeautifulSoup(html_source, 'html.parser')

def use_xpath(soup):
    return etree.HTML(str(soup))

def xpath_get_text(dom, xpath):
    texto = dom.xpath(xpath)[0].text
    return texto

In [ ]:
soup_imovel_url = goto_imovel_url('url')
dom = use_xpath(soup_imovel_url)

### Imobiliária 1/3

In [130]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes = pd.DataFrame(data=None, columns=df_columns)
df_acomodacoes_hashcode = pd.DataFrame(data=None, columns=df_columns)

site_index = 0

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

driver = webdriver.Chrome(options=chrome_options)
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/div').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/ul/li[11]/a').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[7]/a/div').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*50)
        print('Página {}'.format(pag))
        print('-'*50)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        soup_acomodacoes = soup.find_all('div', {'class': 'list__card jetgrid__col--6 jetgrid__col--md-8 jetgrid__col--sm-12 jetgrid__col--xs-24'})
        
        # Coletar dados de cada anúncio
        for num_acomodacao, acomodacao in enumerate(soup_acomodacoes):
            imovel_url = ss_sites[site_index] + acomodacao.find('a')['href']
            cod = acomodacao.find('p', {'class': 'list__reference'}).get_text()
            nome = acomodacao.find('p', {'class': 'list__type'}).get_text()
            bairro = acomodacao.find('p', {'class': 'list__address'}).get_text()
            aluguel = acomodacao.find('div', {'class': 'list__price'}).get_text().strip()
            print(f'Acomodação: {num_acomodacao+1}')
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel_url = goto_imovel_url(imovel_url)
            time.sleep(3.5)
            #dom = use_xpath(soup_imovel_url)
            
            # Adicionar manualmente
            webbrowser.open_new_tab(imovel_url)
            
            condominio = float(input('Digite o valor do condomínio: '))
            area = int(input('Digite a área: '))
            num_quartos = int(input('Digite o número de quartos: '))
            num_banheiros = int(input('Digite o número de banheiros: '))
            vaga_garagem = input('Digite se há vaga de garagem: ')
            descricao = soup_imovel_url.find('p', {'class': 'card__text'}).get_text()
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            print('*'*30)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').click()
        pag += 1
        time.sleep(3)
        
except:
    print('Fim')


--------------------------------------------------
Página 1
--------------------------------------------------
Acomodação: 1
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Acomodação: 2
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Acomodação: 3
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 950,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: 1
******************************
Acomodação: 4
Chacara Antonieta - Limeira/SP
Locação:  R$ 850,00
Digite o valor do condomínio: 0
Digite a área: 30
Digite o núm

### Imobiliária 2/3

In [110]:
soup_test = goto_imovel_url('url')
dom_test = use_xpath(soup)

In [133]:
def scroll_command():
    js_scroll_command = "function getElementByXpath(path) {return document.evaluate(path, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;}"
    js_scroll_command += "\nconst theElement = getElementByXpath('/html/body/main/section[1]/div[2]/div')"
    #js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTo(0, 2000 * scroll_step);}"
    js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTop = node.scrollHeight;}"
    js_scroll_command += f"\nscrollToBottom(theElement);"
    return js_scroll_command

In [137]:
site_index = 1

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

######################################
# Acessar página de kitnets
######################################
time.sleep(3)
# Select tipo
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/button').click()
# kitnet
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/div/ul/li[7]/a').click()
# Botão search
driver.find_element_by_id('submit-main-search-form').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1
num_imoveis = 0
anuncios_encontrados = 0
anuncios = ''

# Coletar dados da página estaticamente com BeautifulSoup
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')
time.sleep(2)

# Coletar o número de imóveis encontrados
num_imoveis = soup.find('section', {'class': 'page-busca-resultado'}).find('h3').get_text()
num_imoveis = re.findall('[0-9]+', num_imoveis)
num_imoveis = int(num_imoveis[0])
print(f'num_imoveis: {num_imoveis}')

# Dar scroll até encontrar todos os imóveis
while anuncios_encontrados < num_imoveis:
    print()
    print('-'*50)
    print('Página {}'.format(pag))
    print('-'*50)
    pag += 1
    
    # Dar scroll
    driver.execute_script(scroll_command())
    time.sleep(2)
    
    # Recarregar página do soup
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(3)
    
    # Coletar os anuncios exibidos
    anuncios = soup.find('div', {'id': 'imovel-boxes'}).find_all('div', {'class': 'col-xs-12 imovel-box-single'})
    anuncios_encontrados = len(anuncios)
    
    print(f'anuncios_encontrados : {anuncios_encontrados}')
    print(anuncios[0].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())
    print(anuncios[-1].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())

num_imoveis: 43

--------------------------------------------------
Página 1
--------------------------------------------------
anuncios_encontrados : 24
Cód.: 36639
Cód.: 11199

--------------------------------------------------
Página 2
--------------------------------------------------
anuncios_encontrados : 36
Cód.: 36639
Cód.: 11183

--------------------------------------------------
Página 3
--------------------------------------------------
anuncios_encontrados : 43
Cód.: 36639
Cód.: 51101


In [138]:
# Coletar dados de cada anúncio
for num_acomodacao, acomodacao in enumerate(anuncios):
    imovel_url = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a')['href']
    cod = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text()
    nome = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a').get_text()
    bairro = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('a')[-1].get_text()
    aluguel = acomodacao.find('span', {'class': 'thumb-price'}).get_text()
    print(f'Acomodação: {num_acomodacao+1}')
    print(bairro)
    print(aluguel)
    
    # Pegar os demais dados
    soup_imovel = goto_imovel_url(imovel_url)
    time.sleep(1)
    
    soup_amenities = soup_imovel.find('div', {'class': 'property-amenities'})
    condominio = 0 # Sem dados
    try:
        area = soup_amenities.find('div', {'id': 'amenity-area-privativa'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        area = int(input('Digite a área: '))
    try:
        num_quartos = soup_amenities.find('div', {'id': 'amenity-dormitorios'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        num_quartos = int(input('Digite o número de quartos: '))
    try:
        num_banheiros = soup_amenities.find('div', {'id': 'amenity-banheiros'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        num_banheiros = int(input('Digite o número de banheiros: '))
    vaga_garagem = 'null' # Sem dados
    descricao = soup_imovel.find('section', {'id': 'clb-descricao'}).find('div', {'class': 'col-xs-12 col-sm-12 col-md-7 col-lg-8'}).find('p').get_text()
    
    # Adicionar registro
    df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                           [ss_sites[site_index], 
                                            imovel_url,
                                            cod,
                                            nome, 
                                            bairro,
                                            aluguel,
                                            condominio,
                                            area,
                                            num_quartos,
                                            num_banheiros,
                                            vaga_garagem,
                                            descricao])
    df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                    [hash_data(ss_sites[site_index]),
                                                     hash_data(imovel_url),
                                                     hash_data(cod), 
                                                     hash_data(nome), 
                                                     bairro,
                                                     aluguel,
                                                     condominio,
                                                     area,
                                                     num_quartos,
                                                     num_banheiros,
                                                     vaga_garagem,
                                                     descricao])
    print('*'*60)

Acomodação: 1
Jardim Cidade Universitária I - Limeira/Sp 
Sob consulta
************************************************************
Acomodação: 2
Jardim Morro Azul - Limeira/Sp 
R$ 500,00
************************************************************
Acomodação: 3
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 4
Jardim Cidade Universitária I - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 5
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 6
Jardim São Paulo - Limeira/Sp 
R$ 780,00
************************************************************
Acomodação: 7
Jardim Cidade Universitária I - Limeira/Sp 
R$ 780,00
Digite o número de quartos: 1
Digite o número de banheiros: 1
************************************************************
Acomodação: 8
Jardim Cidade Universitária I - Limeira/Sp 
R$ 800,00
*************

In [140]:
df_acomodacoes.to_csv('../Data/Imobiliarias/acomodacoes.csv', index=False)
df_acomodacoes_hashcode.to_csv('../Data/acomodacoes_hashcode.csv', index=False)

In [139]:
df_acomodacoes_hashcode

,imob,imovel_url,codigo,nome,bairro,aluguel,condominio,area,quartos,banheiros,vaga_garagem,descricao
0,ef23a7e1738f4b316011bbdd88e514a2,0f91776f479d37e4cae87f2024e335f5,8ea6bceec892ab9d2467eb912385a1a9,fc5f4e72ac82ea901f0f4f77ab0e9281,Jardim Paulista - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,"Kitnet com um dormitório, banheiro, cozinha, á..."
1,ef23a7e1738f4b316011bbdd88e514a2,912f2851a966813353f1ed1a4d7872f5,50f5a26c0b4c05c14d551542eee9cd82,ca19a963cee27eb10567c5523549eeae,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,50 m2,1,1,null,"Imóvel com sala e cozinha conjugados, sendo a..."
2,ef23a7e1738f4b316011bbdd88e514a2,8908dfa6147f15aa381fb9ef4ea8e3d7,87547b6f14ce4c758c1abf4490af8414,ca19a963cee27eb10567c5523549eeae,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,50 m2,1,1,null,"Imóvel com sala e cozinha conjugados, sendo a..."
3,ef23a7e1738f4b316011bbdd88e514a2,47f3e2bcb4dd9dcf4fc279b4c6e8542d,2c8c9e9121145fbc37ef67c0a5ea3869,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,Imóvel muito bem localizado. Conta com sala e...
4,ef23a7e1738f4b316011bbdd88e514a2,21ca795d8f3fc4478746970ee9e386da,358c95a5fb745563272054e6f2a14985,ca19a963cee27eb10567c5523549eeae,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,50 m2,1,1,null,"Imóvel com cozinha com gabinete, armários, mes..."
5,ef23a7e1738f4b316011bbdd88e514a2,bc551ab3987404ca6db1142eb1954c40,2f8542abf804d897430c0c9a84b1991e,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,Imóvel muito bem localizado! Conta com sala e...
6,ef23a7e1738f4b316011bbdd88e514a2,b3f8850fa40b0fe1573bab491166a0e7,7ef0dfe78be61f5fd8392d3948fb860c,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,"Imóvel com: geladeira, fogão, gabinete e mesa ..."
7,ef23a7e1738f4b316011bbdd88e514a2,0fc6a848ea777d994497b7dcfd4f41b6,5416c9b0aba5eb43b419b107d3975a76,ca19a963cee27eb10567c5523549eeae,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,50 m2,1,1,null,Imóvel muito bem localizado Conta com sala e ...
8,ef23a7e1738f4b316011bbdd88e514a2,0ba85ac82e015afbcfa90463af2a8aec,e3ac2c5e0d51230d756de5db78c7452d,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,Imóvel em frente à portaria principal da Unica...
9,ef23a7e1738f4b316011bbdd88e514a2,1ea21866cc1dce17743884bf5f572aec,8fb0a0798b0f5b56b77aaf7df6294525,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,"R$ 1.000,00",0.0,0 m2,1,1,null,Imóvel próximo à Unicamp e também com entrada ...
